In [2]:
import pickle
import numpy as np

with open("data/ability_joints.pkl", "rb") as f:
    data = pickle.load(f)

print(np.array(data['data']).shape)
print(data['meta_data']['joint_names'])

joint_data = np.array(data['data'])

(621, 10)
['index_q1', 'index_q2', 'middle_q1', 'middle_q2', 'pinky_q1', 'pinky_q2', 'ring_q1', 'ring_q2', 'thumb_q1', 'thumb_q2']


In [3]:
import torch,sys
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,get_idxs,np_uv

xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)
env.set_geom_color(
    rgba                = (0.2,0.2,0.2,0.9),
    body_names_to_color = [
        'robot1/end_link','robot1/index_L1','robot1/index_L2','robot1/index_tip',
        'robot1/middle_L1','robot1/middle_L2','robot1/middle_tip',
        'robot1/ring_L1','robot1/ring_L2','robot1/ring_tip',
        'robot1/pinky_L1','robot1/pinky_L2','robot1/pinky_tip',
        'robot1/thumb_L1','robot1/thumb_L2','robot1/thumb_tip',
        'robot2/end_link','robot2/index_L1','robot2/index_L2','robot2/index_tip',
        'robot2/middle_L1','robot2/middle_L2','robot2/middle_tip',
        'robot2/ring_L1','robot2/ring_L2','robot2/ring_tip',
        'robot2/pinky_L1','robot2/pinky_L2','robot2/pinky_tip',
        'robot2/thumb_L1','robot2/thumb_L2','robot2/thumb_tip'],
)

right_hand_joint_names = ['robot2/' + i for i in data['meta_data']['joint_names']]
right_hand_idxs = get_idxs(env.joint_names,right_hand_joint_names)

print ("Done.")

Done.


In [4]:
env.init_viewer(viewer_title='Dual Arms',viewer_width=800,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

q_init = np.zeros(env.model.nq)
q_init[21] = -np.pi/2

env.forward(q=q_init)
tick = 0
seq_len = joint_data.shape[0]

while(env.is_viewer_alive() and tick < seq_len):
    env.forward(q=joint_data[tick],joint_idxs=right_hand_idxs)
    env.render()
    env.plot_T(p=np.zeros(3),R=np.eye(3),label='Tick: %d/%d'%(tick,seq_len),PLOT_AXIS=False)
    
    tick += 1
    if tick == seq_len:tick = 0

env.close_viewer()

Pressed ESC
Quitting.
